# 

# Quarterioni - SciML: Chapter 3.2.1-3.2.4

In [ ]:
%pip install numpy matplotlib polars scipy scikit-learn

## Least square regression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import polars as pl

### Experience (E)

In [ ]:
try:
    import pyodide_http

    pyodide_http.patch_all()
except ImportError:
    pass
df = pl.read_csv(
    "https://raw.githubusercontent.com/ywchiu/riii/refs/heads/master/data/house-prices.csv"
)
df

In [ ]:
# x_labels = ["SqFt", "Bedrooms", "Bathrooms"]
x_labels = ["SqFt"]
y_label = "Price"

### Candidate model

In [ ]:
def f(x):
    return m * x + q

### Training (least square)

In [ ]:
x2_sum = df.select(pl.col(*x_labels).pow(2).sum()).to_numpy()[0]
x_sum = df.select(pl.col(*x_labels).sum()).to_numpy()[0]
x_y_sum = df.select((pl.col(*x_labels) * pl.col(y_label)).sum()).to_numpy()[0]
y_sum = df.select(pl.col(y_label).sum()).to_numpy()[0]
N = [len(df)]

In [ ]:
A = np.array([np.hstack([x2_sum, x_sum]), np.hstack([x_sum, N])])
b = np.hstack([x_y_sum, y_sum])
print(A, b)

In [ ]:
m, q = scipy.linalg.solve(A, b)

### Measure performance

In [ ]:
x = df[x_labels].to_numpy()
y_pred = f(x)
y = df[y_label].to_numpy()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, y_pred)
ax.scatter(x, y)
plt.show()

In [ ]:
MSE = ((y - y_pred) ** 2).sum() / len(y)
print(MSE)

In [ ]:
RMSE = np.sqrt(MSE)
print(RMSE)

## Performance metrics

Give $N$ observations and $(\mathbf{y}_i, \hat{\mathbf{y}}_i) \in \mathbb{R}^m$ the performance is
$$
P = \frac{1}{N} d_m (\mathbf{y}_i, \hat{\mathbf{y}}_i)
$$

for suitable metrix $d_m: \mathbb{R}^m \times \mathbb{R}^m \mapsto \mathbb{R}$

### MSE (Mean Squared Error)

$$ 
d_m = \frac{1}{m} \sum_{i=1}^{m} \| \mathbf{y}_i - \hat{\mathbf{y}}_i \|_2^2
$$ 

**Advantages:**
- Differentiable, making it suitable for gradient-based optimization.
  
**Disadvantages:**
- Sensitive to outliers because of the squaring parameter
- The unit is changed because of the squaring term


### RMSE (Root Mean Squared Error)

$$ 
d_m = \sqrt{\frac{1}{m} \sum_{i=1}^{m} \| \mathbf{y}_i - \hat{\mathbf{y}}_i \|_2^2}
$$ 

**Advantages:**
- Same unit
- Differentiable, making it suitable for gradient-based optimization.
  
**Disadvantages:**
- Sensitive to outliers because of the squaring parameter
- Derivative is more complex which might require more flops to compute gradient

## MAE 

$$
d_m = \frac{1}{m} \sum_{i=1}^{m} \| \mathbf{y}_i - \hat{\mathbf{y}}_i \|_1
$$ 

**Advantages:**
- Same unit
- Less senstitive to outliers
  
**Disadvantages:**
- Not differentiable at zero which makes it challenging for gradient-based optimization methods


### Summary

- Use MSE if you want to penalize larger errors more heavily and prefer a mathematically simple and computationally efficient loss function.
- Use RMSE if you need an error metric on the same scale as your target variable and still want to penalize larger errors, but less severely than MSE.
- Use MAE if you prefer a loss function that is robust to outliers, easily interpretable, and treats all errors equally.


## Classification

Example taken from https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html

In [ ]:
import matplotlib.pyplot as plt

from sklearn import datasets, svm
from sklearn.inspection import DecisionBoundaryDisplay


# import some data to play with
iris = datasets.load_iris()
# Take the first two features. We could avoid this by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
models = (
    svm.SVC(kernel="linear", C=C),
    svm.LinearSVC(C=C, max_iter=10000),
    svm.SVC(kernel="rbf", gamma=0.7, C=C),
    svm.SVC(kernel="poly", degree=3, gamma="auto", C=C),
)
models = (clf.fit(X, y) for clf in models)

# title for the plots
titles = (
    "SVC with linear kernel",
    "LinearSVC (linear kernel)",
    "SVC with RBF kernel",
    "SVC with polynomial (degree 3) kernel",
)

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X[:, 0], X[:, 1]

for clf, title, ax in zip(models, titles, sub.flatten()):
    disp = DecisionBoundaryDisplay.from_estimator(
        clf,
        X,
        response_method="predict",
        cmap=plt.cm.coolwarm,
        alpha=0.8,
        ax=ax,
        xlabel=iris.feature_names[0],
        ylabel=iris.feature_names[1],
    )
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

plt.show()

## Machine learning models

We assume there is a releation between the input and output data.

$$
f : x \mapsto y
$$

We need a rich enough hypothesis space.


What are wrong with e.g polynomials or fourier series (which we know are dense in $C(\Omega)$)?

1D: 

$$
f(x_1) = a_0 + a_1x_1 + a_2x_1^2 + \cdots
$$

2D (same degree polynomial):


$$
f(x_1, x_2) = a_{00} + a_{10}x_1 + a_{01}x_2 + a_{20}x_1^2 + a_{02}x_2^2   + a_{11}x_1x_2 + \cdots
$$


Number of parmameters of a degree $n$ polynomial with $k$ variables is

$$
\binom{k + n}{n}
$$

In [ ]:
k = 2
n = 2
scipy.special.binom(k + n, n)

In [ ]:
k = 100
n = 100
scipy.special.binom(k + n, n)

For neural networks we also have the [universal approximation theorem](https://www.deep-mind.org/2023/03/26/the-universal-approximation-theorem/#Universal_Approximation_Theorem).
For NN we can get away with fewer parameters. See also https://math.uchicago.edu/~may/REU2018/REUPapers/Guilhoto.pdf